# Bee Colony Optimization Algorithm

## Source Code

### Class definition

In [40]:
#Import the random and numpy library
import random
import numpy as np

#Class definition of Bee
class Bee:
    #Initialisation of parameters for bee
    def __init__(self, num_dimensions):
        self.solution = np.random.uniform(low=-5.12, high=5.12, size=num_dimensions)
        self.fitness = None
    
    #Evaluation of the fitness function
    def evaluate_fitness(self, fitness_function):
        self.fitness = fitness_function(self.solution)
#Class Definition of Bee Colony Optimization algorithm      
class BeeColonyOptimization:
    #Initialization of parameters
    def __init__(self, fitness_function, num_dimensions, num_bees, num_iterations, elite_sites, selected_sites, other_sites, elite_prob, other_prob):
        self.fitness_function = fitness_function
        self.num_dimensions = num_dimensions
        self.num_bees = num_bees
        self.num_iterations = num_iterations
        self.elite_sites = elite_sites
        self.selected_sites = selected_sites
        self.other_sites = other_sites
        self.elite_prob = elite_prob
        self.other_prob = other_prob
        self.population = []
        self.best_solution = None
        self.best_fitness = None
    
    #Initialize the bees for this algorithm
    def initialize_bees(self):
        for i in range(self.num_bees):
            bee = Bee(self.num_dimensions)
            bee.evaluate_fitness(self.fitness_function)
            self.population.append(bee)
    
    #Deploy the employee bee to evaluate solution
    def send_employee_bees(self):
        for i, bee in enumerate(self.population):
            neighbor = self.get_neighbor(bee)
            bee.evaluate_fitness(self.fitness_function)
            neighbor.evaluate_fitness(self.fitness_function)
            #Update the solution and fitness through the neighbour
            if neighbor.fitness > bee.fitness:
                bee.solution = neighbor.solution
                bee.fitness = neighbor.fitness
    
    #Deploy onlooker bees for comparison of solution
    def send_onlooker_bees(self):
        #fitness check
        fitnesses = [bee.fitness for bee in self.population]
        #Probability evaluation
        probabilities = fitnesses / np.sum(fitnesses)
        for i in range(self.selected_sites):
            selected_bee = np.random.choice(self.population, p=probabilities)
            neighbor = self.get_neighbor(selected_bee)
            selected_bee.evaluate_fitness(self.fitness_function)
            neighbor.evaluate_fitness(self.fitness_function)
            #Update the values of onlooker bees through its neighbour
            if neighbor.fitness > selected_bee.fitness:
                selected_bee.solution = neighbor.solution
                selected_bee.fitness = neighbor.fitness
    
    #Deploy scout bees to observe performance
    def send_scout_bees(self):
        for bee in self.population:
            if bee.fitness is None:
                bee.evaluate_fitness(self.fitness_function)
            if bee.fitness is None or np.isnan(bee.fitness):
                bee.solution = np.random.uniform(low=-5.12, high=5.12, size=self.num_dimensions)
                bee.evaluate_fitness(self.fitness_function)
                
    #To evaluate the probability of neighbourhood
    def get_neighbor(self, bee):
        site = random.uniform(0, 1)
        if site < self.elite_prob:
            neighbor = self.get_elite_neighbor(bee)
        elif site < self.elite_prob + self.other_prob:
            neighbor = self.get_other_neighbor(bee)
        else:
            neighbor = self.get_random_neighbor(bee)
        return neighbor
    
    #Discriminate between bees and elite bees
    def get_elite_neighbor(self, bee):
        elite_bees = sorted(self.population, key=lambda x: x.fitness, reverse=True)[:self.elite_sites]
        if bee in elite_bees:
            elite_bees.remove(bee)
        neighbor = random.choice(elite_bees)
        return neighbor

    #Sort out the neighbor
    def get_other_neighbor(self, bee):
        other_bees = sorted(self.population, key=lambda x: x.fitness, reverse=True)[self.elite_sites:self.elite_sites + self.other_sites]
        neighbor = random.choice(other_bees)
        return neighbor
    
    def get_random_neighbor(self, bee):
        neighbor = Bee(self.num_dimensions)
        return neighbor
    
    def run(self):
        self.initialize_bees()

        # initialize best solution and fitness
        self.best_solution = self.population[0].solution
        self.best_fitness = self.population[0].fitness

        # start optimization iterations
        for iteration in range(self.num_iterations):
            # send employee bees
            self.send_employee_bees()

            # send onlooker bees
            self.send_onlooker_bees()

            # send scout bees
            self.send_scout_bees()

            # check if new best solution is found
            for bee in self.population:
                if bee.fitness > self.best_fitness:
                    self.best_solution = bee.solution
                    self.best_fitness = bee.fitness

            # print progress every 10 iterations
            #if iteration % 10 == 0:
                print(f"Iteration {iteration}: Best fitness = {self.best_fitness:.6f}")

        return self.best_solution, self.best_fitness


### Fitness Function

In [ ]:

import numpy as np

def fitness_function(x):
    return np.sum(x**4)

### Evaluation 

In [41]:
# Define the bounds of the search space
bounds = np.array([[-5, 5], [-5, 5]])

# Create a random solution within the search space
x = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(1, 2))

# Evaluate the fitness function on the random solution
fitness = fitness_function(x)

# Calculate the dimension of the fitness value
fitness_dim = np.array([fitness]).ndim

print(f"Fitness value: {fitness}")
print(f"Fitness dimension: {fitness_dim}")

#Running an instance of the Bee Colony Optimization Algorithm
aco = BeeColonyOptimization(fitness_function, num_dimensions=3, num_bees=100, num_iterations=100, elite_sites=2, selected_sites=20, other_sites=28, elite_prob=0.9, other_prob=0.1)
best_solution, best_fitness = aco.run()
print(f"Best solution: {best_solution}")
print(f"Best fitness: {best_fitness}")

Fitness value: 75.55119050466203
Fitness dimension: 1
Iteration 0: Best fitness = 1135.813021
Iteration 0: Best fitness = 1135.813021
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness = 1163.041454
Iteration 0: Best fitness 

Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Best fitness = 1163.041454
Iteration 11: Be

Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 19: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Best fitness = 1163.041454
Iteration 20: Be

Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Best fitness = 1163.041454
Iteration 30: Be

Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Best fitness = 1163.041454
Iteration 40: Be

...

Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Best fitness = 1163.041454
Iteration 60: Be

Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 68: Best fitness = 1163.041454
Iteration 69: Best fitness = 1163.041454
Iteration 69: Best fitness = 1163.041454
Iteration 69: Best fitness = 1163.041454
Iteration 69: Best fitness = 1163.041454
Iteration 69: Best fitness = 1163.041454
Iteration 69: Be

Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 77: Best fitness = 1163.041454
Iteration 78: Be

Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 88: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Best fitness = 1163.041454
Iteration 89: Be

Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 98: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Best fitness = 1163.041454
Iteration 99: Be